# DATA COLLECTION

In [32]:
import numpy as np
import pandas as pd
import PyPDF2 # PDF reader 
import re # Regex
import string
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

In [7]:
PDF = open('data/LEGITEXT000006074228.pdf', 'rb')  

In [8]:
Reader = PyPDF2.PdfFileReader(PDF)  
Reader.numPages  # Nombre de pages dans le document

516

In [9]:
Page = Reader.getPage(0)  # Choisir une page
print(Page.extractText()) # Afficher la page choisie

Code de la route. - Dernière modification le 01 juin 2022 - Document généré le 31 mai 2022
Code de la route
Partie législative
Livre 1er : Dispositions générales
Titre 1er : Définitions.
Article L110-1
 
Pour l'application du présent code, les termes, ci-après ont le sens qui leur est donné dans le présent article :
 
 
1° Le terme "véhicule à moteur" désigne tout véhicule terrestre pourvu d'un moteur de propulsion, y compris
les trolleybus, et circulant sur route par ses moyens propres, à l'exception des véhicules qui se déplacent sur
rails ;
 
 
2° Le terme "remorque" désigne tout véhicule destiné à être attelé à un autre véhicule.
 
Article L110-2
 
La définition des voiries nationales, départementales et communales est fixée aux articles L. 121-1, L. 122-1,
L. 123-1, L. 131-1, L. 141-1, L. 151-1 et L. 161-1 du code de la voirie routière ci-après reproduits :
 
" Art. L. 121-1.-Les voies du domaine public routier national sont :
 
1° Les autoroutes ;
 
2° Les routes nationales.
 
Le

In [10]:
# Récupère tout le texte dans une seule string
code_brut = ''
for i in range(Reader.numPages):
    Pages = Reader.getPage(i)
    code_brut += Pages.extractText()


In [11]:
len(code_brut) # Nombre de caractères

1311177

# Data Cleaning

In [12]:
code_brut

'Code de la route. - Dernière modification le 01 juin 2022 - Document généré le 31 mai 2022\nCode de la route\nPartie législative\nLivre 1er : Dispositions générales\nTitre 1er : Définitions.\nArticle L110-1\n \nPour l\'application du présent code, les termes, ci-après ont le sens qui leur est donné dans le présent article :\n \n \n1° Le terme "véhicule à moteur" désigne tout véhicule terrestre pourvu d\'un moteur de propulsion, y compris\nles trolleybus, et circulant sur route par ses moyens propres, à l\'exception des véhicules qui se déplacent sur\nrails ;\n \n \n2° Le terme "remorque" désigne tout véhicule destiné à être attelé à un autre véhicule.\n \nArticle L110-2\n \nLa définition des voiries nationales, départementales et communales est fixée aux articles L. 121-1, L. 122-1,\nL. 123-1, L. 131-1, L. 141-1, L. 151-1 et L. 161-1 du code de la voirie routière ci-après reproduits :\n \n" Art. L. 121-1.-Les voies du domaine public routier national sont :\n \n1° Les autoroutes ;\n \n

In [13]:
code = code_brut.replace("\n", " ") # retrait passage à la ligne
code = re.sub("\'", " ", code) # retrait des apostrophes : \'
code = code.replace("Code de la route. - Dernière modification le 01 juin 2022 - Document généré le 31 mai 2022","") # retrait bas de page
# retirer : Parties/Livres/Titres => on peut retrouver le ces infos grâce à l'intitulé de l'article
code

' Code de la route Partie législative Livre 1er : Dispositions générales Titre 1er : Définitions. Article L110-1   Pour l application du présent code, les termes, ci-après ont le sens qui leur est donné dans le présent article :     1° Le terme "véhicule à moteur" désigne tout véhicule terrestre pourvu d un moteur de propulsion, y compris les trolleybus, et circulant sur route par ses moyens propres, à l exception des véhicules qui se déplacent sur rails ;     2° Le terme "remorque" désigne tout véhicule destiné à être attelé à un autre véhicule.   Article L110-2   La définition des voiries nationales, départementales et communales est fixée aux articles L. 121-1, L. 122-1, L. 123-1, L. 131-1, L. 141-1, L. 151-1 et L. 161-1 du code de la voirie routière ci-après reproduits :   " Art. L. 121-1.-Les voies du domaine public routier national sont :   1° Les autoroutes ;   2° Les routes nationales.   Le domaine public routier national est constitué d un réseau cohérent d autoroutes et de ro

In [14]:
articles = re.findall(r"(Article \w\d*-?\d.*?(?=Article))",code) # split la string par article pour en faire une liste d'articles

In [15]:
articles

['Article L110-1   Pour l application du présent code, les termes, ci-après ont le sens qui leur est donné dans le présent article :     1° Le terme "véhicule à moteur" désigne tout véhicule terrestre pourvu d un moteur de propulsion, y compris les trolleybus, et circulant sur route par ses moyens propres, à l exception des véhicules qui se déplacent sur rails ;     2° Le terme "remorque" désigne tout véhicule destiné à être attelé à un autre véhicule.   ',
 'Article L110-2   La définition des voiries nationales, départementales et communales est fixée aux articles L. 121-1, L. 122-1, L. 123-1, L. 131-1, L. 141-1, L. 151-1 et L. 161-1 du code de la voirie routière ci-après reproduits :   " Art. L. 121-1.-Les voies du domaine public routier national sont :   1° Les autoroutes ;   2° Les routes nationales.   Le domaine public routier national est constitué d un réseau cohérent d autoroutes et de routes d intérêt national ou européen. Des décrets en Conseil d Etat, actualisés tous les dix

In [16]:
# depuis la liste d'articles vers un dataframe
dict_articles = {}
for i in range(len(articles)):
    dict_articles[i] = articles[i]

data = pd.DataFrame.from_dict(dict_articles, orient='index',
                       columns=[ 'article_base'])

In [17]:
# return le numero de l'article
def article_number(x): 
    return x[:17]
data['article_number'] = data['article_base'].apply(lambda x : article_number(x))

In [18]:
# return l'article
def article_context(x): 
    return x[17:]
data['article_context'] = data['article_base'].apply(lambda x : article_context(x))

In [19]:
# lowercase 
def article_lower(x):
    return x.lower()
data['article_lowered'] = data['article_context'].apply(lambda x : article_lower(x))

In [20]:
# retirer les chiffres
def remove_numbers(x):
    return ''.join([i for i in x if not i.isdigit()])
data['article_wo_numbers'] = data['article_lowered'].apply(lambda x : remove_numbers(x))


In [34]:
# retirer Punctuation (accents /caractères spéciaux ) 
def remove_punctuation(x):
    for punctuation in string.punctuation :
        x = x.replace(punctuation,"")
    return x
data['article_wo_punctuation'] = data['article_wo_numbers'].apply(lambda x : remove_punctuation(x))

In [86]:
# Retirer les Stopwords / Frequent Words 
stop_words = set(stopwords.words('french'))
def remove_stopwords(x):
    word_tokens = word_tokenize(x)     
    x = [word for word in word_tokens if not word in stop_words] 
    x = [word for word in x if len(word)>1] # retrait des caractères spéciaux restants
    return x
data['article_wo_stopwords'] = data['article_wo_punctuation'].apply(lambda x : remove_stopwords(x))
data

,article_base,article_number,article_context,article_lowered,article_wo_numbers,article_wo_punctuation,article_wo_stopwords
0,Article L110-1 Pour l application du présent...,Article L110-1,"Pour l application du présent code, les termes...","pour l application du présent code, les termes...","pour l application du présent code, les termes...",pour l application du présent code les termes ...,"[application, présent, code, termes, ciaprès, ..."
1,Article L110-2 La définition des voiries nat...,Article L110-2,"La définition des voiries nationales, départem...","la définition des voiries nationales, départem...","la définition des voiries nationales, départem...",la définition des voiries nationales départeme...,"[définition, voiries, nationales, départementa..."
2,Article L110-3 Les routes à grande circulati...,Article L110-3,"Les routes à grande circulation, quelle que so...","les routes à grande circulation, quelle que so...","les routes à grande circulation, quelle que so...",les routes à grande circulation quelle que soi...,"[routes, grande, circulation, quelle, apparten..."
3,Article L121-1 Le conducteur d un véhicule e...,Article L121-1,Le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,"[conducteur, véhicule, responsable, pénalement..."
4,Article L121-2 Par dérogation aux dispositio...,Article L121-2,Par dérogation aux dispositions de l article L...,par dérogation aux dispositions de l article l...,par dérogation aux dispositions de l article l...,par dérogation aux dispositions de l article l...,"[dérogation, dispositions, article, titulaire,..."
...,...,...,...,...,...,...,...
1144,Article A121-2 I.- Lorsque ces information...,Article A121-2,I.- Lorsque ces informations sont adressées ...,i.- lorsque ces informations sont adressées ...,i.- lorsque ces informations sont adressées ...,i lorsque ces informations sont adressées pa...,"[lorsque, informations, adressées, lettre, rec..."
1145,Article A121-3 I.- Lorsque ces information...,Article A121-3,I.- Lorsque ces informations sont adressées ...,i.- lorsque ces informations sont adressées ...,i.- lorsque ces informations sont adressées ...,i lorsque ces informations sont adressées de...,"[lorsque, informations, adressées, façon, déma..."
1146,Article A143-1 Sont applicables en Nouvell...,Article A143-1,"Sont applicables en Nouvelle-Calédonie, en P...","sont applicables en nouvelle-calédonie, en p...","sont applicables en nouvelle-calédonie, en p...",sont applicables en nouvellecalédonie en pol...,"[applicables, nouvellecalédonie, polynésie, fr..."
1147,Article A325-12 Le modèle de fiche descripti...,Article A325-12,Le modèle de fiche descriptive de l état du v...,le modèle de fiche descriptive de l état du v...,le modèle de fiche descriptive de l état du v...,le modèle de fiche descriptive de l état du v...,"[modèle, fiche, descriptive, état, véhicule, e..."


In [ ]:
# Lemmatizer 

In [87]:
list(data['article_wo_stopwords'][:1])

[['application',
  'présent',
  'code',
  'termes',
  'ciaprès',
  'sens',
  'donné',
  'présent',
  'article',
  'terme',
  'véhicule',
  'moteur',
  'désigne',
  'tout',
  'véhicule',
  'terrestre',
  'pourvu',
  'moteur',
  'propulsion',
  'compris',
  'trolleybus',
  'circulant',
  'route',
  'moyens',
  'propres',
  'exception',
  'véhicules',
  'déplacent',
  'rails',
  'terme',
  'remorque',
  'désigne',
  'tout',
  'véhicule',
  'destiné',
  'être',
  'attelé',
  'autre',
  'véhicule']]

# Feature Engineering

In [ ]:
# Parties/Livres/Titres
# Voc Richness
# Digit caracter ratio